In [4]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.isidewith.com/polls/popular'
response = requests.get(url)

issues_votes = []

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    polls = soup.find_all('div', class_='poll')
    
    for poll in polls:
        # Correctly identifying the issue name
        img_div = poll.find('div', class_='img')
        if img_div:
            p_tag = img_div.find('p')
            if p_tag:
                span_tag = p_tag.find('span')
                if span_tag:
                    issue_name = span_tag.text.strip()

                    # Adjusting logic for finding vote count
                    count_div = poll.find('div', class_='count')
                    if count_div:
                        vote_text = count_div.text.strip().split()[0].replace(',', '')
                        vote_count = int(vote_text) if vote_text.isdigit() else 0
                        
                        # Appending issue name and vote count to the list
                        issues_votes.append((issue_name, vote_count))

    # Sorting and extracting the top 100 issues based on vote count
    issues_votes.sort(key=lambda x: x[1], reverse=True)
    top_issues = issues_votes[:100]

    iteration = 1
    for issue, votes in top_issues:
        print(f"Issue {iteration}: {issue}, Votes: {votes}")
        iteration = iteration + 1
else:
    print(f"Failed to fetch webpage: Status code {response.status_code}")



Issue 1: Abortion, Votes: 36422084
Issue 2: Gun Control, Votes: 36407755
Issue 3: Immigration Healthcare, Votes: 35084464
Issue 4: Obamacare, Votes: 35048729
Issue 5: Gay Marriage, Votes: 34695076
Issue 6: Marijuana, Votes: 33666341
Issue 7: Minimum Wage, Votes: 27225743
Issue 8: Terrorism, Votes: 26670374
Issue 9: Common Core, Votes: 25008260
Issue 10: Climate Change, Votes: 24713635
Issue 11: Student Loans, Votes: 24572468
Issue 12: Oil Drilling, Votes: 21665712
Issue 13: Voter Fraud, Votes: 19320820
Issue 14: Immigration, Votes: 17960950
Issue 15: Death Penalty, Votes: 16815198
Issue 16: Planned Parenthood Funding, Votes: 16755158
Issue 17: Drug Policy, Votes: 16653057
Issue 18: Border Security, Votes: 16107125
Issue 19: Affirmative Action, Votes: 15442805
Issue 20: Immigrant Laborers, Votes: 15275785
Issue 21: Fracking, Votes: 14694673
Issue 22: Equal Pay, Votes: 14456923
Issue 23: Government Mandates, Votes: 14446116
Issue 24: In-State Tuition, Votes: 14066110
Issue 25: Government

In [5]:
# Extract just the issue names from each tuple
issues = [issue[0] for issue in top_issues]

# Print out the list to verify
for issue in issues:
    print(issue)
print(issues)

Abortion
Gun Control
Immigration Healthcare
Obamacare
Gay Marriage
Marijuana
Minimum Wage
Terrorism
Common Core
Climate Change
Student Loans
Oil Drilling
Voter Fraud
Immigration
Death Penalty
Planned Parenthood Funding
Drug Policy
Border Security
Affirmative Action
Immigrant Laborers
Fracking
Equal Pay
Government Mandates
In-State Tuition
Government Spending
United Nations
Term Limits
Patriot Act
Solitary Confinement for Juveniles
Space Exploration
Medicaid
Religious Freedom Act
Criminal Voting Rights
Farm Subsidies
Alternative Energy
Israel
Mandatory Vaccinations
GMO Labels
Drones
Euthanasia
First Amendment
Immigrant Assimilation
No-Fly List Gun Control
Nuclear Energy
Police Body Cameras
Net Neutrality
Illegal Immigrant Detainment
Pension Reform
Gerrymandering
NSA Surveillance
Labor Unions
Property Taxes
Confederate Flag
Welfare Drug Testing
Border Wall
Drug Price Regulation
Military Spending
Foreign Assassination
Right of Foreigners to Vote
NSA Domestic Surveillance
Mandatory Militar

In [ ]:
'''from ntscraper import Nitter
import pandas as pd

# Initialize the Nitter scraper
scraper = Nitter(log_level=1)
'''



In [ ]:
'''from ntscraper import Nitter
import pandas as pd
import time

results = []

def scrape_data_for_term(issue):
    try:
        tweets = scraper.get_tweets(issue, mode='term', number=5, language='en')
        if isinstance(tweets, list):
            for tweet in tweets:
                if isinstance(tweet, dict):
                    results.append({
                        'issue': issue,
                        'tweet': tweet.get('content', ''),
                        'likes': tweet.get('likeCount', 0),
                        'retweets': tweet.get('retweetCount', 0),
                        # Add other fields as needed
                    })
        else:
            print(f"No valid tweets returned for {issue}")
    except Exception as e:
        print(f"Error while fetching data for {issue}: {str(e)}")
        time.sleep(60)  # Sleep to avoid hitting rate limit quickly

for issue in issues:
    scrape_data_for_term(issue)

df = pd.DataFrame(results)
df.to_csv('scraped_tweets.csv', index=False)
print("Data scraping complete. Results saved to 'scraped_tweets.csv'.")
'''

In [ ]:
'''import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler("E25G8XB4oorLGCTdo3U8ZsXfn", "0jBPTkYVU6zsMvcUAf1RqVFxbvI6OeiXuVFzhUpMpCIRvHOzKX")
auth.set_access_token("1328482358044012544-2q2pCvQR9GiZv80nqEHKGuCvVSDdRB", "8VoFpz5A2f824soL0adWjSxfml4220weD4GPPgqIBSeRh")

api = tweepy.API(auth)

# Collect tweets
for issue in issues:
    tweets = api.search_tweets(q=issue, count=1)  # Adjust count as needed
    for tweet in tweets:
        # Here you can collect the data you need
        print(tweet.text, tweet.favorite_count, tweet.retweet_count, tweet.reply_count, tweet.author.followers_count)

'''

In [8]:
#correlation = df[['likeCount', 'replyCount', 'quoteCount', 'retweetCount']].corrwith(df['viewCount'])

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Load your dataset
df = pd.read_csv("TweetsDataIssues.csv")

# Calculate correlations
correlations = df[['likeCount', 'replyCount', 'quoteCount', 'retweetCount', 'viewCount']].corr()
print("Correlation with View Count:\n", correlations['viewCount'])

# Prepare data
X = df[['likeCount', 'replyCount', 'quoteCount', 'retweetCount']]
y = df['viewCount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model = LinearRegression()
model.fit(X_train, y_train)

# Get coefficients
coefficients = model.coef_
print("Regression Coefficients:", coefficients)

# Perform cross-validation
scores = cross_val_score(model, X, y, cv=5)
print("Cross-validation scores:", scores.mean())


Correlation with View Count:
 likeCount       0.551383
replyCount      0.505307
quoteCount      0.619434
retweetCount    0.475442
viewCount       1.000000
Name: viewCount, dtype: float64
Regression Coefficients: [  42.73814368   76.64729704 1397.03089274  -69.95816526]
Cross-validation scores: 0.46837311931604725
